In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import time
from pickle import load
import tensorflow as tf
from tensorflow import keras

tf.random.set_seed(42)
K = keras.backend

from sklearn.preprocessing import StandardScaler,PowerTransformer
import math
from scipy.stats import multivariate_normal
from scipy.stats import norm
import os
import pickle
from helper import *

import scipy.optimize


2023-04-13 17:09:36.958457: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
k = 8
J = 32
save_time_step = 0.005
h=1 
c=10
b=10


# Prepare data for training with differing F values

In [3]:
train1 = np.load("../../data/truth_run/climate_change_exp/train_set_1.npy")
train2 = np.load("../../data/truth_run/climate_change_exp/train_set_2.npy")
train3 = np.load("../../data/truth_run/climate_change_exp/train_set_3.npy")
f20 = np.load("../../data/truth_run/training_dataset.npy")
train4 = np.concatenate([f20, 20*np.ones(shape=(f20.shape[0],k,1))],axis=2)[:200000]

def u_deriver(x_train,F,save_time_step=save_time_step):
    u_t = -np.roll(x_train[:-1,:],1,axis=1) * (np.roll(x_train[:-1,:],2,axis=1) - np.roll(x_train[:-1,:],-1,axis=1)) - x_train[:-1,:] + F -(x_train[1:,:] - x_train[:-1,:])/save_time_step
    return u_t

u_t_train_1 = u_deriver(train1[:,:,0],train1[0,0,-1])
u_t_train_2 = u_deriver(train2[:,:,0],train2[0,0,-1])
u_t_train_3 = u_deriver(train3[:,:,0],train3[0,0,-1])
u_t_train_4 = u_deriver(train4[:,:,0],train4[0,0,-1])

x_combo = np.ravel(np.concatenate([train1[:-1,:,0],train2[:-1,:,0],train3[:-1,:,0],train4[:-1,:,0]],axis=0)).reshape(-1,1)


u_combo = np.ravel(np.concatenate([u_t_train_1,u_t_train_2,u_t_train_3,u_t_train_4],axis=0)).reshape(-1,1)



scaler1 = StandardScaler()
scaler2 = StandardScaler()

x_t_scaled = scaler1.fit_transform(x_combo)
u_t_scaled = scaler2.fit_transform(u_combo)

In [18]:
u_t_original = (np.concatenate([u_t_train_1,u_t_train_2,u_t_train_3,u_t_train_4],axis=0))

In [72]:
x_t_scaled.shape

(3999992, 1)

In [ ]:
codings_size = 1 + 1 + 2*16


## AR1 parameters ##

In [9]:
generator = keras.models.load_model("gan_generator_final.h5")

In [10]:
input_dataset3 = [x_t_scaled,np.random.normal(size=(x_t_scaled.shape[0],codings_size))]

In [11]:
u_t_pred = generator(input_dataset3)

In [12]:
u_t_pred_unscaled_gagne_gan = scaler2.inverse_transform(u_t_pred)

In [16]:
u_t_original.shape

(3999992,)

In [19]:
residual = u_t_original - u_t_pred_unscaled_gagne_gan.reshape(-1,k)

In [20]:
residual_shaped = residual.reshape(-1,order="f")


In [21]:
phi = np.corrcoef(residual_shaped[:-1],residual_shaped[1:])[1,0]
phi

0.7083031902758682

In [22]:
sigma = (1-phi**2)**0.5
sigma

0.7059083443656317